# Smooth Monotonic Networks: Plots and tables
This file reproduces the plots, statistical tests, and LaTeX tables in the manuscript.

In [ ]:
import numpy as np

import torch 
import torch.nn as nn

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.isotonic import IsotonicRegression

from scipy.stats import wilcoxon

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset

from tqdm.notebook import tnrange

from xgboost import XGBRegressor


You need to set the following prefix variable for reading the result files produced by ``MonotonicNNFullyMonotoneExperiments.ipynb`` and ``UCI validation.ipynb``.

In [ ]:
path = "./"

## Univariate experiments

In [ ]:
fn = path + "univariate.npz"
data = np.load(fn, allow_pickle = True)
print(data.files)

mse_train = np.swapaxes(data['MSE_train'], 1, 2)
mse_test = np.swapaxes(data['MSE_test'], 1, 2)
mse_clip = np.swapaxes(data['MSE_clip'], 1, 2)

In [ ]:
labels = (r"MM", r"SMM", r"XG", r"XG$_{\text{val}}$", r"Iso", r"HLL", r"LMN$^{\text{s}}$", r"LMN$^{\text{l}}$" )

print(mse_train.shape, len(labels))
print(mse_test.sum(), mse_clip.sum())

In [ ]:
functions = ("\\fsq", "\\fsqrt", "\\fsig")
alpha = .001  
sig_test = np.zeros((len(functions), len(labels)))
sig_train = np.zeros((len(functions), len(labels)))
for f in range(len(functions)):
    f_smm_test = mse_test[f,:,1]
    f_smm_train = mse_train[f,:,1]
    for m in range(len(labels)):
        if(m!=1):
            f_alt = mse_test[f,:,m] 
            pv = wilcoxon(f_smm_test, f_alt).pvalue
            sig_test[f,m]=(pv < alpha)
            f_alt = mse_train[f,:,m] 
            pv = wilcoxon(f_smm_train, f_alt).pvalue
            sig_train[f,m]=(pv < alpha)
            #print(pv< 0.01/len(title), pv)
print(sig_test)
print(sig_train)

In [ ]:
print(data['no_params'])

In [ ]:
np.set_printoptions(precision=4, floatmode='fixed')
meds =  np.median(mse_test, axis=1)
meds_min = np.min(meds, axis=1)

scale=1000
for f, f_name in enumerate(functions):
    print(f_name, '&', end=' ')
    for m in range(len(labels)):
        med = meds[f, m]
        if (med==meds_min[f]):
            print("\\low{"+"{:.2f}".format(med*scale)+"}", end='')
        else:
            print("{:.2f}".format(med*scale), end='')
        if sig_test[f,m]:
            print("\sigdif", end='')
        if m < len(labels)-1:
            print(' & ', end='')
        else:
            print(' \\\\')
            

In [ ]:
scale=1000
for f, f_name in enumerate(functions):
    print(f_name, '&', end=' ')
    for m in range(len(labels)):
        print("{:.2f}".format(np.median(mse_train[f,:,m])*scale), end='')
        if m < len(labels)-1:
            print(' & ', end='')
        else:
            print(' \\\\')


In [ ]:
# methods = ['monotonic', 'smooth', 'xgboost', 'xgboost_val', 'iso', 'hll', 'lip_small', 'lip']

fn_tag = ""

plt.rcParams['text.usetex'] = True
plt.rc('text.latex', preamble=r'\usepackage{amsmath}')

colors=('b', 'g', 'r', 'c', 'm', "k")
title = (r"\Large$f_{\text{sq}}$", r"\Large$f_{\text{sqrt}}$", r"\Large$f_{\text{sig}}$")
fig, ax = plt.subplots(2, 3, sharex=True, figsize=(10, 6), layout='constrained')
for i in range(3):
    ax[0,i].set_title(title[i])
    ax[0,i].boxplot(mse_train[i], labels=labels);
    ax[1,i].boxplot(mse_test[i], labels=labels);
ax[0,0].set_ylabel(r'training MSE')
ax[1,0].set_ylabel(r'test MSE (w/o noise)')
plt.savefig(path + "bar1D"+ fn_tag + ".pdf")
plt.show();

In [ ]:


plt.rcParams['text.usetex'] = True
plt.rc('text.latex', preamble=r'\usepackage{amsmath}')

colors=('b', 'g', 'r', 'c', 'm', "k", 'b','y')
title = (r"\Large$f_{\text{sq}}$", r"\Large$f_{\text{sqrt}}$", r"\Large$f_{\text{sig}}$")
fig, ax = plt.subplots(1, 3, sharex=True, figsize=(10, 3), layout='constrained')
for i in range(3):
    ax[i].set_title(title[i])
    ax[i].boxplot(mse_train[i], labels=labels);
ax[0].set_ylabel(r'training MSE')
plt.savefig(path + "bar1D_train"+ fn_tag + ".pdf")
plt.show();

In [ ]:
trial = 11
fig, ax = plt.subplots(2, 3, figsize=(10, 6), layout='constrained')
for i in range(3):
    ax[0,i].set_title(title[i])
    ax[0,i].plot(data['X_test'][i, trial], data['Y_test'][i, trial], '--', color='0.8')
    ax[1,i].plot(data['X_test'][i, trial], data['Y_test'][i, trial], '--', color='0.8')
    for j in [2,3,4]:
        ax[0,i].plot(data['X_test'][i, trial], data['O_test'][i, j, trial], label=labels[j], color=colors[j])
    for j in [0,1,5,7]:
        ax[1,i].plot(data['X_test'][i, trial], data['O_test'][i, j, trial], label=labels[j], color=colors[j])
    ax[0,0].legend(handlelength=4)
    ax[1,0].legend(handlelength=4)
    ax[0,0].set_ylabel(r'$y$')
    ax[1,0].set_ylabel(r'$y$')
    ax[1,i].set_xlabel(r'$x$')
    x1, x2, y1, y2 = .7, .9, .875, .975
    
    if i>0:
        axi = inset_axes(ax[0, i], width="40%", height="20%", loc=4, borderpad=1)
        for j in [2,3,4]:
            axi.plot(data['X_test'][i, trial], data['O_test'][i, j, trial], label=labels[j], color=colors[j])
        axi.set_xlim(x1, x2)
        axi.set_ylim(y1, y2)
        axi.tick_params(labelleft=False, labelbottom=False)
        mark_inset(ax[0, i], axi, loc1=2, loc2=4, fc="none", ec="0.5")
        axi = inset_axes(ax[1, i], width="40%", height="20%", loc=4, borderpad=1)
        for j in [0,1,5,7]:
            axi.plot(data['X_test'][i, trial], data['O_test'][i, j, trial], label=labels[j], color=colors[j])
        axi.set_xlim(x1, x2)
        axi.set_ylim(y1, y2)
        axi.tick_params(labelleft=False, labelbottom=False)
        mark_inset(ax[1, i], axi, loc1=2, loc2=4, fc="none", ec="0.5")
plt.savefig(path + "example" + str(trial) + "_1D"+ ".pdf")

## Multivariate experiments

In [ ]:
fn = path + "multivariate.npz"
data = np.load(fn, allow_pickle = True)
print(data.files)

mse_train = np.swapaxes(data['MSE_train'], 1, 2)
mse_test = np.swapaxes(data['MSE_test'], 1, 2)

meds =  np.median(mse_test, axis=1)
meds_min = np.min(meds, axis=1)

print(mse_train.shape)

In [ ]:
#methods = ['smooth', 'xgboost', 'xgboost_val', 'xgboost2', 'xgboost2_val','lattice', 'lattice_plus', 'lip_small', 'lip']
#['smooth', 'xgboost', 'xgboost_val', 'xgboost2', 'xgboost2_val','lattice', 'lattice_plus']
labels = (r"SMM", r"XG$^{\text{s}}$", r"XG$^{\text{s}}_{\text{val}}$", r"XG$^{\text{l}}$", r"XG$^{\text{l}}_{\text{val}}$",
          r"HLL$^{\text{s}}$", r"HLL$^{\text{l}}$", r"LMN$^{\text{s}}$", r"LMN$^{\text{l}}$")
print(len(labels))



plt.rcParams['text.usetex'] = True
plt.rc('text.latex', preamble=r'\usepackage{amsmath}')

colors=('b', 'g', 'r', 'c', 'm', "k")
title = (r"$d=2$", r"$d=4$", r"$d=6$")
fig, ax = plt.subplots(2, 3, sharex=True, figsize=(10, 6), layout='constrained')
for i in range(3):
    ax[0,i].set_title(title[i])
    ax[0,i].boxplot(mse_train[i], labels=labels);
    ax[1,i].boxplot(mse_test[i], labels=labels);
ax[0,0].set_ylabel(r'training MSE')
ax[1,0].set_ylabel(r'test MSE (w/o noise)')
plt.savefig(path + "barMulti"+ ".pdf")
plt.show();

In [ ]:
functions = ("$d=2$", "$d=4$", "$d=6$")
alpha = .001  
ref_index = 0 # reference is SMM, index 0
sig_test = np.zeros((len(functions), len(labels)))
sig_train = np.zeros((len(functions), len(labels)))
for f in range(len(functions)):
    f_smm_test = mse_test[f,:,ref_index] 
    f_smm_train = mse_train[f,:,ref_index]
    for m in range(len(labels)):
        if(m!=ref_index):
            f_alt = mse_test[f,:,m] 
            pv = wilcoxon(f_smm_test, f_alt).pvalue
            sig_test[f,m]=(pv < alpha)
            f_alt = mse_train[f,:,m] 
            pv = wilcoxon(f_smm_train, f_alt).pvalue
            sig_train[f,m]=(pv < alpha)
            #print(pv< 0.01/len(title), pv)

In [ ]:
np.set_printoptions(precision=4, floatmode='fixed')
scale=1000

for f, f_name in enumerate(functions):
    print(f_name, '&', end=' ')
    for m in range(len(labels)):
        med = meds[f, m]
        if (med==meds_min[f]):
            print("\\low{"+"{:.2f}".format(med*scale)+"}", end='')
        else:
            print("{:.2f}".format(med*scale), end='')
        if sig_test[f,m]:
            print("\sigdif", end='')
        if m < len(labels)-1:
            print(' & ', end='')
        else:
            print(' \\\\')

In [ ]:
for f, f_name in enumerate(functions):
    print(f_name, '&', end=' ')
    for m in range(len(labels)):
        print("{:.2f}".format(np.median(mse_train[f,:,m])*scale), end='')
        if m < len(labels)-1:
            print(' & ', end='')
        else:
            print(' \\\\')

## UCI tasks

In [ ]:
methods = ("\\MLPSMM", "\\SMM", "\\XG", "\\HLL", "\\LMNs", "\\LMNl")
tasks = ("\energyOne", "\energyTwo", "\qsar", "\concrete")
data_energy1 = np.load(path + "energy-y1-results-val.npz", allow_pickle = True)
data_energy2 = np.load(path + "energy-y2-results-val.npz", allow_pickle = True)
data_qsar = np.load(path + "qsar-results-val.npz", allow_pickle = True)
data_concrete = np.load(path + "concrete-results-val.npz", allow_pickle = True)
print(*data_energy1)
print(*data_energy2)
print(*data_qsar)
print(*data_concrete)

In [ ]:
for i in range(len(methods)):
    print(methods[i])
    print(data_energy1['no_params'][i])
    print(data_energy2['no_params'][i])
    print(data_qsar['no_params'][i])
    print(data_concrete['no_params'][i])

In [ ]:
means = []
no_params = []
mse_test = data_energy1['MSE_test']
means.append(np.mean(mse_test, axis=1))
no_params.append(data_energy1['no_params'])
mse_test = data_energy2['MSE_test']
means.append(np.mean(mse_test, axis=1))
no_params.append(data_energy2['no_params'])
mse_test = data_qsar['MSE_test']
means.append(np.mean(mse_test, axis=1))
no_params.append(data_qsar['no_params'])
mse_test = data_concrete['MSE_test']
means.append(np.mean(mse_test, axis=1))
no_params.append(data_concrete['no_params'])

In [ ]:
scale = 100
for i in methods:
    print(" & \multicolumn{2}{c}{", i, "}", end='')
print(" \\\\")
for i, (mean, no) in enumerate(zip(means, no_params)):
    print(tasks[i], end='')
    for j, k in zip(mean, no):
        if (j==min(mean)):
            print(" & ", "\\low{"+"{:.4f}".format(j*scale)+'}', end='')
        else:
            print(" & ", "{:.4f}".format(j*scale), end='')
        print(" & ", "{:d}".format(int(k)), end='')
    print(" \\\\")

# Eval hyperparameters

In [ ]:
fn = path + "hyper.npz"
data = np.load(fn, allow_pickle = True)
print(data.files)

In [ ]:
d = data['MSE_test']
print(d.shape)

In [ ]:
print(np.min(d, axis=(1,2,3)))
#np.argmin(d, axis=(1,2,3))

In [ ]:
functions = ("\\fsq", "\\fsqrt", "\\fsig")
K_values = (2, 4, 6, 8)
beta_values = (-3., -2., -1., 0., 1.)



In [ ]:
m = np.median(d, axis=3)
mv = np.min(m, axis=(1,2))
for beta_id, beta in enumerate(beta_values):
    print(" &", beta, end='')
print('\\\\')
for task_id, task in enumerate(functions):
    print(" & \multicolumn{5}{c}{", task, "}\\\\")
    
    for K_id, K in enumerate(K_values):
        print ( K, end='')
        for beta_id, beta in enumerate(beta_values):
            v = m[task_id, K_id, beta_id]
            if v==mv[task_id]:
                print(" & \\low{", "{:.4f}".format(v*1000), end='}')
            else:
                print(" & ", "{:.4f}".format(v*1000), end='')
        print('\\\\')